In [1]:
#importing all the packages needed
import dash
from dash import dcc, html, callback, Output, Input
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import requests
import io

In [2]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
response = requests.get(url)
spacex_df = pd.read_csv(url)

spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [6]:
# Create the Dash app
app = dash.Dash(__name__)

min_value = 0
max_value = 10000

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1(
        children='SpaceX Launch Records Dashboard',
        style={
            'textAlign': 'center',
            'color': '#503D36',
            'fontSize': '24px'
        }
    ),
    
    # Dropdown menu
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
        ],
        value='ALL',
        placeholder="Select a site",
        searchable=True
    ),
    
    # Title for the range slider
    html.Label(
        children='Payload Range (Kg)',
        style={
            'fontSize': '18px',
            'marginBottom': '10px'
        }
    ),
    
    # Range slider
    dcc.RangeSlider(
        id='payload-slider',
        min=0, max=10000, step=1000,
        marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
        value=[min_value, max_value]
    ),
    
    dcc.Graph(id='success-pie-chart'),
    dcc.Graph(id='success-payload-scatter-chart')
])

# Callback to update the pie chart
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Use the entire dataframe for all sites
        filtered_df = spacex_df
        fig = px.pie(
            filtered_df, 
            values='class', 
            names='Launch Site', 
            title='Total Success Launches By Site'
        )
    else:
        # Filter the dataframe by the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        
        # Count the occurrences of each class (0 and 1)
        class_counts = filtered_df['class'].value_counts().reset_index()
        class_counts.columns = ['class', 'count']  # Rename columns for clarity
        
        # Create a pie chart for success/failure distribution at the selected site
        fig = px.pie(
            class_counts, 
            values='count', 
            names='class', 
            title=f'Success vs. Failure for {entered_site}'
        )
    return fig

# Callback to update the scatter chart
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_chart(entered_site, payload_range):
    # Check for correct column names and data types
    if 'Payload Mass (kg)' not in spacex_df.columns or 'Booster Version' not in spacex_df.columns or 'Launch Site' not in spacex_df.columns:
        raise ValueError('Required columns are missing in the dataframe')
    
    # Filter data based on payload range
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
        (spacex_df['Payload Mass (kg)'] <= payload_range[1])
    ]
    
    # If ALL sites are selected
    if entered_site == 'ALL':
        fig = px.scatter(
            filtered_df, x='Payload Mass (kg)', y='class',
            color='Booster Version Category',  # Color by booster version category
            title='Correlation between Payload and Success for All Sites'
        )
    else:
        # Filter for the selected site
        site_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(
            site_df, x='Payload Mass (kg)', y='class',
            color='Booster Version Category',  # Color by booster version category
            title=f'Correlation between Payload and Success for site {entered_site}'
        )
    
    return fig

# Run the app server
if __name__ == '__main__':
    app.run_server(debug=True)
